<a href="https://colab.research.google.com/github/CodeHunterOfficial/TimeSeriesForecast_2024-2025/blob/main/%D0%A2%D0%B5%D0%BC%D0%B0%205.%20%D0%9F%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D1%80%D1%8F%D0%B4%D0%BE%D0%B2%20%D1%81%20%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E%20%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D0%BE%D0%B3%D0%BE%20%D0%B1%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3%D0%B0/%D0%A2%D0%B5%D0%BC%D0%B0_5_2_%D0%9F%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D1%80%D1%8F%D0%B4%D0%BE%D0%B2_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B1%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3%D0%B0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тема 5.2. Прогнозирование временных рядов с помощью градиентного бустинга

#5.3.  Градиентный бустинг


В предыдущих разделах мы изучили способы объединения базовых алгоритмов в ансамбль с помощью бэггинга, включая построение случайных лесов из решающих деревьев. Теперь рассмотрим другой метод композиции — градиентный бустинг.

При обучении случайного леса каждое дерево строится независимо от остальных. В отличие от этого, бустинг представляет собой процесс построения последовательной линейной комбинации алгоритмов, где каждый последующий алгоритм направлен на уменьшение ошибки текущего ансамбля.

Градиентный бустинг на решающих деревьях (Gradient Boosting on Decision Trees, GBDT) отлично справляется с "табличными" данными, которые часто бывают неоднородными. Примерами таких данных являются характеристики пользователя (возраст, пол, среднее количество поисковых запросов в день, число заказов такси и т.д.). Этот метод эффективно выявляет нелинейные зависимости в разнообразных данных.

Хотя деревья решений подходят для объединения в ансамбли в различных задачах, GBDT часто оказывается лучшим выбором благодаря своим высоким показателям в машинном обучении и промышленных задачах, таких как:
- поисковое ранжирование;
- рекомендательные системы;
- таргетированная реклама;
- прогнозирование погоды;
- выбор пунктов назначения для такси и другие.

Однако в задачах с однородными данными, такими как тексты, изображения, звук или видео, бустинг может не проявить себя наилучшим образом. В этих сценариях нейросетевые подходы часто демонстрируют более высокое качество.

Важно отметить, что возможность использовать в качестве базовых алгоритмов не только деревья решений, но и другие модели, например, линейные модели, реализована в пакете XGBoost. Однако следует учитывать, что при использовании линейных моделей композиция также будет представлять собой линейную комбинацию, что снижает способность ансамбля эффективно моделировать нелинейные зависимости в данных. Поэтому в данном разделе мы ограничимся рассмотрением бустинга на решающих деревьях.

## Литература
1. https://education.yandex.ru/handbook/ml/article/gradientnyj-busting
2. https://explained.ai/gradient-boosting/L2-loss.html
3.